In [1]:
import os, sys

PROJECT_DIR = os.path.abspath(os.path.join(os.getcwd(), '.'))
print(PROJECT_DIR)
sys.path.append(PROJECT_DIR)

/workspaces/street_tree


In [2]:
import joblib
import torch
import pandas as pd
import numpy as np
from fastapi import FastAPI, UploadFile, File
from pydantic import BaseModel
from src.modeling import TabularNN
from src.config import PATH_MODELS, PATH_PROCESSED

2025-03-23 14:46:25,551 - src.utils - INFO - Logger is ready
2025-03-23 14:46:25,554 - src.utils - INFO - This is a test log message.
2025-03-23 14:46:26,436 - src.utils - INFO - PROJECT_DIR: /workspaces/street_tree
2025-03-23 14:46:26,437 - src.utils - INFO - PATH_LOGS: /workspaces/street_tree/logs
2025-03-23 14:46:26,438 - src.utils - INFO - PATH_DATA: /workspaces/street_tree/data
2025-03-23 14:46:26,439 - src.utils - INFO - PATH_RAW: /workspaces/street_tree/data/raw
2025-03-23 14:46:26,439 - src.utils - INFO - PATH_PROCESSED: /workspaces/street_tree/data/processed
2025-03-23 14:46:26,440 - src.utils - INFO - PATH_MODELS: /workspaces/street_tree/models
2025-03-23 14:46:26,441 - src.utils - INFO - PATH_REPORTS: /workspaces/street_tree/reports
2025-03-23 14:46:26,441 - src.utils - INFO - PATH_DOCS: /workspaces/street_tree/docs
2025-03-23 14:46:26,442 - src.utils - INFO - PATH_SRC: /workspaces/street_tree/src
2025-03-23 14:46:26,443 - src.utils - INFO - output_path: /workspaces/street_t

In [3]:
app = FastAPI()

In [4]:
# Пути к моделям и энкодерам
MODEL_PATH = f"{PATH_MODELS}/tabular_model.pth"
SCALER_PATH = f"{PATH_MODELS}/scaler.pkl"
ENCODERS_PATH = f"{PATH_MODELS}/label_encoders.pkl"


In [5]:
TARGET_ENCODER_PATH = f"{PATH_MODELS}/label_encoders_target.pkl"
target_mapping = joblib.load(TARGET_ENCODER_PATH)
inverse_target_mapping = {v: k for k, v in target_mapping.items()}
inverse_target_mapping

{0: 'Poor', 1: 'Fair', 2: 'Good'}

In [6]:
# Пути к моделям и энкодерам
MODEL_PATH = f"{PATH_MODELS}/tabular_model.pth"
SCALER_PATH = f"{PATH_MODELS}/scaler.pkl"
ENCODERS_PATH = f"{PATH_MODELS}/label_encoders.pkl"

In [7]:
# Load the checkpoint to get the parameters
checkpoint = torch.load(MODEL_PATH)
input_dim = checkpoint['input_dim']
hidden_dims = checkpoint['hidden_dims']
output_dim = checkpoint['output_dim']
target_mapping = checkpoint['target_mapping']

# Create dummy data with the correct shape
dummy_X = pd.DataFrame(np.zeros((3, input_dim)))
dummy_y = pd.Series([0, 1, 2]) 

# Initialize the model using dummy data
loaded_model = TabularNN(dummy_X, dummy_y, dummy_X, dummy_y, hidden_dims=hidden_dims)
loaded_model.load_state_dict(checkpoint['model_state_dict'])
loaded_model.eval()

print("Model loaded successfully!")


loaded_model

2025-03-23 14:46:26,513 - src.utils - INFO - Unique classes during training: [0 1 2]
2025-03-23 14:46:26,514 - src.utils - INFO - Output dimension during training: 3


Model loaded successfully!


TabularNN(
  (model): Sequential(
    (0): Linear(in_features=27, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=64, out_features=3, bias=True)
  )
)

In [8]:
# Обратное отображение для целевого признака
scaler = joblib.load(SCALER_PATH)
label_encoders = joblib.load(ENCODERS_PATH)
target_mapping = joblib.load(TARGET_ENCODER_PATH)
inverse_target_mapping = {v: k for k, v in target_mapping.items()}
inverse_target_mapping

{0: 'Poor', 1: 'Fair', 2: 'Good'}

In [9]:
label_encoders

{'spc_latin': LabelEncoder(),
 'steward': LabelEncoder(),
 'guards': LabelEncoder(),
 'user_type': LabelEncoder(),
 'borough': LabelEncoder(),
 'nta': LabelEncoder()}

In [10]:
import pandas as pd
from src.model import TreeData
from src.config import PATH_PROCESSED
import json


# Пример использования:
df = pd.read_csv(f"{PATH_PROCESSED}/test.csv")
df = df.drop('health', axis=1)
df.columns = [col.lower().replace(' ', '_') for col in df.columns]
data_json = df.to_dict(orient='records')
json_data = json.dumps(data_json)

tree_data_list = TreeData.from_dataframe(json_data)

In [11]:
# Extract tree_id and data from TreeData objects
data = []
tree_ids = []
for tree_data in tree_data_list:
    tree_ids.append(tree_data.tree_id)
    data.append(tree_data.dict())

# Create DataFrame from data
df_reconstructed = pd.DataFrame(data, index=tree_ids)

# Set tree_id as index
df_reconstructed.index.name = 'tree_id'

df_reconstructed.head()

/tmp/ipykernel_61238/1157747548.py:6: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  data.append(tree_data.dict())


,tree_id,block_id,tree_dbh,curb_loc,spc_latin,steward,guards,sidewalk,user_type,problems,...,brch_other,postcode,borough,cncldist,st_assem,st_senate,nta,boro_ct,latitude,longitude
tree_id,,,,,,,,,,,,,,,,,,,,,
536325,536325,415591,56,0,Fraxinus,None,Unsure,0,NYC Parks Staff,NoProblem,...,0,10306,Staten Island,50,62,24,SI24,5027900,40.571808,-74.144252
247341,247341,335158,16,1,Acer platanoides,None,Unsure,0,NYC Parks Staff,"RootOther,TrunkOther,BranchOther",...,1,11426,Queens,23,33,11,QN43,4161700,40.728383,-73.710717
427681,427681,202590,2,1,Amelanchier,None,Unsure,0,NYC Parks Staff,NoProblem,...,0,11208,Brooklyn,37,54,18,BK83,3118800,40.680291,-73.865729
442929,442929,501795,15,1,Gleditsia triacanthos var. inermis,None,Unsure,1,TreesCount Staff,Stones,...,0,10459,Bronx,17,85,32,BX27,2012701,40.826623,-73.893234
696579,696579,306543,32,1,Quercus palustris,None,Unsure,1,Volunteer,Stones,...,0,11367,Queens,24,27,16,QN37,4080900,40.732411,-73.817595


In [12]:
from src.preprocessing import load_and_encode_categorical, split_problems

data = split_problems(df_reconstructed, created_columns=False)


data = load_and_encode_categorical(data, list(set(label_encoders.keys())), PATH_MODELS)
data.drop('tree_id', axis=1, inplace=True)
# Выгрузим нормальзованные данные
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler = joblib.load(f'{PATH_MODELS}/scaler.pkl')
print("Scaler loaded from scaler.pkl")

data = scaler.transform(data)



2025-03-23 14:46:32,771 - src.utils - INFO - {'stones', 'branchlights', 'sneakers', 'wiresrope', 'trunklights', 'noproblem', 'branchother', 'trunkother', 'rootother', 'metalgrates'} - уникальные проблемы


2025-03-23 14:46:32,909 - src.utils - INFO - Категориальные признаки преобразованы с использованием загруженных LabelEncoders


Scaler loaded from scaler.pkl


In [13]:
predictions = loaded_model.predict(data)
class_labels = [inverse_target_mapping[label] for label in predictions]
class_labels[:5]

['Good', 'Poor', 'Good', 'Good', 'Good']

In [14]:
# Убедимся, что длина предсказаний совпадает с количеством строк в DataFrame
if len(predictions) == len(data) and len(class_labels) == len(data):
    # Добавим предсказания и метки классов в DataFrame
    df_reconstructed['health'] = class_labels
    df_reconstructed.reset_index(inplace=True)

    print(df_reconstructed[['health', 'tree_id']] )
else:
    print("Ошибка: длина предсказаний или меток классов не совпадает с длиной DataFrame.")

      health  tree_id
0       Good   536325
1       Poor   247341
2       Good   427681
3       Good   442929
4       Good   696579
...      ...      ...
65213   Good   169025
65214   Fair   236081
65215   Good   203194
65216   Good    89168
65217   Good   592963

[65218 rows x 2 columns]
